# Analyzing New York City employees' payroll database 

## Data Source: [NYC open data](https://data.cityofnewyork.us/City-Government/Citywide-Payroll-Data-Fiscal-Year-/k397-673e/data)

In [1]:
import pandas as pd
df = pd.read_csv('Citywide_Payroll_Data__Fiscal_Year_.csv')
df.columns = df.columns.str.replace(" ", "_")
df.columns = df.columns.str.replace("-", "_")
df.columns = df.columns.str.lower()
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

/Users/areena.arora/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df.shape

(2864545, 17)

Previous versions of the dataset didn't import all the years correctly, so I'll do these sanity checks a couple of times

In [3]:
df.fiscal_year.value_counts()

2019    592431
2020    590210
2021    573477
2017    562266
2018    546161
Name: fiscal_year, dtype: int64

#### Cleaning the data

The database includes people whose work locations are outside of NYC. For this analyses, I'm only including employees with work locations in NYC boroughs. Since Staten Island wasn't listed (unless included in the "other" location, this is filtered down to Queens, Manhattan, Bronx and Brooklyn.

In [4]:
boroughs = ['QUEENS', 'MANHATTAN', 'BROOKLYN', 'BRONX']

In [5]:
df = df[df.work_location_borough.isin(boroughs)]

In [6]:
df.shape

(2760682, 17)

Note: This reduced the dataset by 103,863 rows.

### A note on the assumptions I'm making here on:

From our reporting, I found that some employees tend to work more overtime as they get closer to retirement. Let's take a closer look here on.

In [12]:
df['agency_start_date'] = pd.to_datetime(df.agency_start_date, errors='coerce')
df['today'] = pd.to_datetime('today')
df['tenure_years'] = (df.today - df.agency_start_date).astype('timedelta64[Y]')

In [13]:
df

,fiscal_year,payroll_number,agency_name,last_name,first_name,mid_init,agency_start_date,work_location_borough,title_description,leave_status_as_of_june_30,base_salary,pay_basis,regular_hours,regular_gross_paid,ot_hours,total_ot_paid,total_other_pay,today,tenure,tenure_years
0,2017,NaN,ADMIN FOR CHILDREN'S SVCS,AARON,TERESA,NaN,2016-03-21,BRONX,CHILD PROTECTIVE SPECIALIST,ACTIVE,"51,315.00",per Annum,"1,825.00","51,709.59",588.00,"22,374.31",639.66,2022-05-07 19:31:01.812351,6.00,6.00
1,2017,NaN,ADMIN FOR CHILDREN'S SVCS,AARONS,CAMELIA,M,2016-08-08,BROOKLYN,CHILD PROTECTIVE SPECIALIST,ACTIVE,"51,315.00",per Annum,"1,595.55","41,960.18",121.75,"3,892.19",108.25,2022-05-07 19:31:01.812351,5.00,5.00
2,2017,NaN,ADMIN FOR CHILDREN'S SVCS,ABDUL,MODUPE,NaN,2008-02-11,BROOKLYN,CHILD PROTECTIVE SPECIALIST,ACTIVE,"54,720.00",per Annum,"1,825.00","56,298.93",54.75,"2,455.88","3,938.75",2022-05-07 19:31:01.812351,14.00,14.00
3,2017,NaN,ADMIN FOR CHILDREN'S SVCS,ABDUL RAHMAN,ABDUL AZIZ,I,2014-10-20,MANHATTAN,CHILD PROTECTIVE SPECIALIST,ACTIVE,"54,720.00",per Annum,"1,825.00","55,346.09",273.00,"11,069.41","1,124.51",2022-05-07 19:31:01.812351,7.00,7.00
4,2017,NaN,ADMIN FOR CHILDREN'S SVCS,ABDULGANIYU,MONSURAT,A,2013-02-04,BRONX,JUVENILE COUNSELOR,ACTIVE,"44,409.00",per Annum,"1,762.00","44,157.49",815.50,"27,878.15","2,019.34",2022-05-07 19:31:01.812351,9.00,9.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2864536,2021,745.00,DEPT OF ED HRLY SUPPORT STAFF,FRATIANNI,MARYANN,C,1999-04-27,MANHATTAN,F/T SCHOOL AIDE,CEASED,17.00,per Hour,0.00,"-57,256.00",0.00,0.00,"5,814.70",2022-05-07 19:31:01.812351,23.00,23.00
2864537,2021,742.00,DEPT OF ED PEDAGOGICAL,LAMBERT,MARISA,M,2005-09-06,MANHATTAN,ASSISTANT PRINCIPAL,ON LEAVE,"130,351.00",per Annum,0.00,"-36,364.44",0.00,0.00,"-15,369.52",2022-05-07 19:31:01.812351,16.00,16.00
2864538,2021,745.00,DEPT OF ED HRLY SUPPORT STAFF,RIVERA,SARAH,M,1997-09-02,MANHATTAN,F/T SCHOOL AIDE,CEASED,17.04,per Hour,0.00,"-58,284.17",0.00,0.00,"4,347.24",2022-05-07 19:31:01.812351,24.00,24.00
2864541,2021,902.00,BRONX DISTRICT ATTORNEY,SIMMONS,NATHANIEL,NaN,1990-07-02,BRONX,SPECIAL ASSISTANT TO THE DISTRICT ATTORNEY,CEASED,"110,000.00",per Annum,-70.00,"-4,207.65",0.00,0.00,"-75,440.00",2022-05-07 19:31:01.812351,31.00,31.00


In [14]:
import statsmodels.formula.api as smf

# YOU CAN ADD FILTERS HERE IF YOU WANT TO LOOK INTO A PARTICULAR AGENCY
# Let's start with no filters
to_model = df # .query("agency_name=='DEPARTMENT OF CORRECTION'")

# title_description
# MODEL y=F(X) - which factors do you want to control for? 
# What do we think should explain the variance in overtime pay
model = smf.ols('ot_hours ~ regular_hours + tenure', data=to_model) 
# note that I added a squared term because it fits better
# https://stackoverflow.com/questions/31978948/python-stats-models-quadratic-term-in-regression

results = model.fit()
display(results.summary())

# FINDING OUTLIERS
# + E (what is still unaccounted for once you have controlled for those factors)
outliers = to_model.assign(
    predicted = results.predict(),
    residulas = results.resid,
    residuals_z = results.resid / results.resid.std()
    )\
    .sort_values(by='residuals_z', ascending=False)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ot_hours   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                  0.333
Method:                 Least Squares   F-statistic:                 6.880e+05
Date:                Sat, 07 May 2022   Prob (F-statistic):               0.00
Time:                        19:36:08   Log-Likelihood:            -1.7350e+07
No. Observations:             2760380   AIC:                         3.470e+07
Df Residuals:                 2760377   BIC:                         3.470e+07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       -14.5109      0.151    -96.040      0.000     -14.807     -14.215
regular_hours     0.1034   8.82e-05   1171.919      0.000       0.103       0.104
tenure            0.5045      0.008     61.266      0.000       0.488       0.521
==============================================================================
Omnibus:                  1872011.418   Durbin-Watson:                   1.182
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         45045085.379
Skew:                           2.940   Prob(JB):                         0.00
Kurtosis:                      21.896   Cond. No.                     2.15e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.15e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

ValueError: Length of values (2760380) does not match length of index (2760682)

In [25]:
df.total_ot_paid.max()

248749.72

In [23]:
df.total_ot_paid.min().round()

-26494.0

In [32]:
df.query('total_ot_paid == 248749.72')

,fiscal_year,payroll_number,agency_name,last_name,first_name,mid_init,agency_start_date,work_location_borough,title_description,leave_status_as_of_june_30,base_salary,pay_basis,regular_hours,regular_gross_paid,ot_hours,total_ot_paid,total_other_pay,today,tenure,tenure_years
2291076,2021,996.00,NYC HOUSING AUTHORITY,PROCIDA,ROBERT,NaN,1987-04-13,BRONX,SUPERVISOR PLUMBER,ACTIVE,387.03,per Day,"1,820.00","100,627.80","2,249.50","248,749.72","7,215.34",2022-05-07 19:31:01.812351,35.00,35.00


In [33]:
df['total_pay'] = df.regular_gross_paid + df.total_ot_paid + df.total_other_pay

In [34]:
df.query('total_ot_paid == 248749.72')

,fiscal_year,payroll_number,agency_name,last_name,first_name,mid_init,agency_start_date,work_location_borough,title_description,leave_status_as_of_june_30,base_salary,pay_basis,regular_hours,regular_gross_paid,ot_hours,total_ot_paid,total_other_pay,today,tenure,tenure_years,total_pay
2291076,2021,996.00,NYC HOUSING AUTHORITY,PROCIDA,ROBERT,NaN,1987-04-13,BRONX,SUPERVISOR PLUMBER,ACTIVE,387.03,per Day,"1,820.00","100,627.80","2,249.50","248,749.72","7,215.34",2022-05-07 19:31:01.812351,35.00,35.00,"356,592.86"


In [39]:
df.groupby(by='agency_name').total_ot_paid.sum().reset_index().sort_values(by='total_ot_paid', ascending=False).head(5)

,agency_name,total_ot_paid
125,POLICE DEPARTMENT,"3,410,660,240.39"
88,FIRE DEPARTMENT,"1,596,842,055.03"
59,DEPARTMENT OF CORRECTION,"981,672,183.09"
65,DEPARTMENT OF SANITATION,"790,873,824.62"
114,NYC HOUSING AUTHORITY,"553,297,085.88"


In [43]:
df.agency_name.value_counts().reset_index().head(100)

,index,agency_name
0,DEPT OF ED PEDAGOGICAL,494409
1,DEPT OF ED PER SESSION TEACHER,466518
2,POLICE DEPARTMENT,292030
3,DEPT OF ED PARA PROFESSIONALS,192268
4,BOARD OF ELECTION POLL WORKERS,183495
...,...,...
95,BROOKLYN COMMUNITY BOARD #10,25
96,QUEENS COMMUNITY BOARD #7,24
97,BRONX COMMUNITY BOARD #9,23
98,QUEENS COMMUNITY BOARD #8,22
